# HW2: Naïve Bayes

In this homework, you'll implement and (very briefly) discuss a bag-of-words Naïve Bayes sentiment classifier—a simple but effective example of a linear classifier.

This assignment is due at the start of class on September 29. When you're done, upload your edited `ipynb` file to NYU Classes.

## Setup

First, let's load the Stanford Sentiment Treebank. If you don't already have it, download it from here: [the train/dev/test Stanford Sentiment Treebank distribution](http://nlp.stanford.edu/sentiment/trainDevTestTrees_PTB.zip), unzip it, and put the resulting folder in the same directory as this notebook. (If you want to put it somewhere else, change `sst_home` below.)

Note: Unlike with k-nearest neighbors, Naïve Bayes evaluation should be quite fast (thousands of examples per second at least), so we don't need to trim down the dev and test sets. 

In [143]:
sst_home = './trees'

import re
import random

def load_sst_data(path):
    
    EASY_LABEL_MAP = {0:0, 1:0, 2:None, 3:1, 4:1}
    
    data = []
    with open(path) as f:
        for i, line in enumerate(f): 
            example = {}
            example['label'] = EASY_LABEL_MAP[int(line[1])]
            if example['label'] is None:
                continue
            
            # Strip out the parse information and the phrase labels
            text = re.sub(r'\s*(\(\d)|(\))\s*', '', line)
            example['text'] = text[1:]
            data.append(example)

    return data
     
training_set = load_sst_data(sst_home + '/train.txt')
dev_set = load_sst_data(sst_home + '/dev.txt')
test_set = load_sst_data(sst_home + '/test.txt')

print dev_set[1]

{'text': "And if you 're not nearly moved to tears by a couple of scenes , you 've got ice water in your veins .", 'label': 1}


## Part 1: Bags of words

Next, let's write a function to convert these sentences into feature vectors. The function template here simply extracts three useless (?) dummy features:

- The number of characters in the review.
- The first letter in the review.
- Whether the letters 'th' appear in the review.

This function depends upon a simple dictionary trick that allows us to reason about features by name rather than by index.

For this classifier, we'll be sticking to bag-of-words features. Delete the existing features, and replace them with bag-of-words features.

In [144]:
import numpy as np
from collections import Counter

In [145]:
def feature_function(datasets):
    ''' A function that converts raw text into bag-of-words features
    
    Parameters
    ----------
    datasets: A list where each entry is one observation, represented as 
        a dictionary with "text" and "label" being keys.
    
    Returns
    -------
    result: A list where each entry is one observation, represented
        as a dictionary with "feature" and "label" being keys. The value of 
        "feature" is a dictionary represented in the format of bag-of-words.
    class_words_occurrences: A dictionary with keys being the class labels
        and value being the number of corresponding words occurrences
    '''
    # a list containing the transformed observations in the datasets
    result = []
    # a counter that counts the total number of word occurrences of each class
    class_counter = Counter()
    # a set containing the vocabulary for the document
    vocabulary = set()
    for obs in datasets:
        # get bag of words features
        obs_bag_of_words = extract_features(obs)
        result.append(obs_bag_of_words)
        # update the vocabulary
        vocabulary.update(obs_bag_of_words['feature'].keys())
        # update the total number of word occurrences for each class
        class_counter[obs["label"]] += np.sum(obs_bag_of_words['feature'].values())
    class_words_occurrences = dict(class_counter)
    num_words = len(vocabulary)
    return result, class_words_occurrences, num_words

def extract_features(observation):
    ''' Extract bag-of-words features from the raw observation represented
    as a dictionary
    '''
    list_of_words = observation["text"].split(" ")
    # get bag of words features
    bag_of_words = count(list_of_words)
    return {"feature": bag_of_words, "label": observation["label"]}
    
def count(words):
    ''' A function that counts the number of occurences of words in a list
    Returns a dictionary representing the words and their occurrences
    '''
    counter = Counter()
    counter.update(words)
    return dict(counter)


## Part 2: Implementing Naïve Bayes

Next, implement a Naïve Bayes classifier that you can train and test on the feature vectors you just extracted. Use Laplace (add-one) smoothing.

In [152]:
class NaiveBayesClassifier:
    ''' A naive bayes classifier using Laplace smoothing
    '''
    def __init__(self):
        # initiate some variables for training and classifying
        self.log_prior = {} # class priors as log probability
        self.log_likelihood = {} # likelihood terms of words as log prob
        self.class_words_occurrences = {} # number of word occurrences for each class
        self.num_words = 0 # total number of words in the vocabulary of the doc
        self.trained = False # already trained or not
        
    def train(self, training_set):
        # get the observation represented as bag-of-words, 
        # the number of all the word occurrences for each class,
        # and the number of words in the vocabulary of training set
        bag_of_words, class_words_occurrences, num_words = (
            feature_function(training_set))
        self.class_words_occurrences = class_words_occurrences
        self.num_words = num_words
        # count the number of observations for each class, to 
        # be used later to calculate the prior of classes
        label_counter = Counter()
        # the number of occurrences of each word for each class
        counters = {}
        
        # initiate a dictionary to store the counters for each class
        for class_label in class_words_occurrences:
            counters[class_label] = Counter()
        # iterate through through all the observations
        for obs in bag_of_words:
            label = obs['label']
            label_counter[label] += 1  # update the count of documents for this class
            features = obs['feature']
            counters[label].update(features)  # update the count of each word for this class

        # update self.prior and self.log_likelihood
        for class_label in class_words_occurrences:
            # calculate the class prior
            self.log_prior[class_label] = (np.log(1.0*label_counter[class_label]
                                                  /len(training_set)))
            # calculate the log likelihood of each word for each class
            # using Laplace smoothing
            log_likelihood = {}
            for word in counters[class_label]:
                log_likelihood[word] = (np.log(1.0*(counters[class_label][word] + 1)/
                                               (class_words_occurrences[class_label] + num_words)))
            self.log_likelihood[class_label] = log_likelihood
        self.trained = True

    def classify(self, example):
        # chech if it's already trained
        if self.trained == False:
            raise Exception("Please train the classifier first.")
        # extract bag-of-words features
        bag_of_words_obs = extract_features(example)
        max_log_sum = -float('inf')
        hypothesis = 0
        for label in self.log_prior:
            log_sum = self.log_prior[label]
            for word in bag_of_words_obs['feature']:
                # if the word has not appreared
                if not word in self.log_likelihood[label]: # use 0+1 as the occurrence of the word
                    log_sum += (np.log(1.0/(self.class_words_occurrences[label] + 
                                            self.num_words)))*bag_of_words_obs['feature'][word]
                else:
                    log_sum += (self.log_likelihood[label][word]*
                                bag_of_words_obs['feature'][word])
            # get the hypothesis with the maximum log sum probability
            if log_sum > max_log_sum:
                max_log_sum = log_sum
                hypothesis = label
        return hypothesis
        

Here's how it's trained:

In [153]:
classifier = NaiveBayesClassifier()
classifier.train(training_set)

Here's how it's called. It returns a label (0 for negative, 1 for positive):

In [154]:
print classifier.classify(dev_set[0])

1


Here's a simple function to evaluate a classifier. It expects a function from example to labels.

In [155]:
def evaluate_classifier(classifier, eval_set):
    correct = 0
    for example in eval_set:
        hypothesis = classifier.classify(example)
        if hypothesis == example['label']:
            correct += 1
    return correct / float(len(eval_set))

This runs the primary evaluation. It'll return accuracy (%). If you've implemented Naïve Bayes correctly, you should see accuracy of greater than 75% on the dev set. The [original Stanford Sentiment Treebank paper](http://nlp.stanford.edu/~socherr/EMNLP2013_RNTN.pdf) reports 81.8% test accuracy with Naïve Bayes.

In [156]:
print evaluate_classifier(classifier, dev_set)

0.792431192661


## Part 3: Questions

Briefly answer each of the questions below.

**Question 1:** Most implementations of Naïve Bayes (hopefully including yours), never actually compute $P(d|c)$, but instead directly compute $\log P(d|c)$. Why is this?

**Answer:** 
1. Since the data matrix can be very sparse, so it's possible that each P(d|c) can be very small, and when we multiply theses independent P's tegether, the product can be so extremely small that the computer cannot accurately represent it with complete precision, which causes underflow. So instead of computing the extremely small probability, we do log probability so that we don't need to care about the precision of extremely small numbers in computer. 
2. By converting multiplication to summation, hopefully the calculation process can be relatively faster.

**Question 2:** In class, we found that a nearest neighbor model built over bag-of-words features barely surpassed 60% accuracy on the dev set. Why is Naïve Bayes so much better at classifying sentences using this same style of feature?

**Answer:**

Theoretically, by directly applying KNN to high dimension data, we get "curse of dimensionality". Since we are using bag of words approach so the dimension can be very high and thus the issue of curse of dimensionality can be quite serious. One critical way to overcome this type of "curse" is to make assumptions about the nature of data distribution. And these assumptions are inherently involved in some parametric models. Since Naive Bayes is essentially a linear classifier and we have very strong assumptions of independence, so the "curse of dimensionality" can be mitigated.

**Question 3:** Do some error analysis---identify three sentences that the model mis-classified, and speculate about ways in which a better (but still realistic) machine learning model trained on this same training set might be able to do better.

**Answer:**
From the error analysis below, we can find that the examples misclassified are usually involve both "good" words and "bad" words. What discriminate good and bad apart in those sentences are not only just dependent on the words, but more dependent on the sementics of the sentence. I guess we can do a little bit feature engineering to deal with this issue, by combining successive words together as an one feature, which means that we can do 2-grams, 3-grams or so. For example, "beautiful, magnificant" can be regarged as "good" indicators by our original model but if we consider the potential "not" before them, then "not beautiful, magnificant" can totally indicates that it's a negative sentiment.

In [151]:
cnt = 0
np.random.shuffle(dev_set)
for example in dev_set:
    hypothesis = classifier.classify(example)
    if hypothesis != example['label']:
        cnt += 1
        print example
        if cnt == 5:
            break
            

{'text': "A broad , melodramatic estrogen opera that 's pretty toxic in its own right .", 'label': 0}
{'text': "All that 's missing is the spontaneity , originality and delight .", 'label': 0}
{'text': 'Scores no points for originality , wit , or intelligence .', 'label': 0}
{'text': "I 've always dreamed of attending Cannes , but after seeing this film , it 's not that big a deal .", 'label': 0}
{'text': "What 's surprising about Full Frontal is that despite its overt self-awareness , parts of the movie still manage to break past the artifice and thoroughly engage you .", 'label': 1}
